In [ ]:
import csv
import torch
import evaluate

from tqdm.auto import tqdm
from torch.optim import AdamW
from torch.utils.data import DataLoader
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler

In [ ]:
model_name = "classla/xlm-roberta-base-multilingual-text-genre-classifier"

In [ ]:
dataset = DatasetDict()

dataset["train"] = load_dataset("csv", data_files="./data/train.csv")["train"]
dataset["valid"] = load_dataset("csv", data_files="./data/valid.csv")["train"]

dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], max_length=512, truncation=True, padding="max_length")

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=77).select(range(1000))
small_valid_dataset = tokenized_datasets["valid"].shuffle(seed=77).select(range(1000))

In [ ]:
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
valid_dataloader = DataLoader(small_valid_dataset, batch_size=8)

In [ ]:
with open("./data/id_to_label.csv", "r") as f:
    reader = csv.DictReader(f)
    id2label = {row["id"]: row["label"] for row in reader}
label2id = {label: id for id, label in id2label.items()}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=18, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [ ]:
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
metric = evaluate.load("accuracy")
model.eval()
for batch in valid_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()